In [ ]:
import numpy as np
import math
from scipy.integrate import solve_ivp
# import matplotlib
# matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

%matplotlib notebook #TODO: fix animation ploting
# %matplotlib widget

In [2]:
def path(p):
    x = 53 + 17.7 * (math.cos(p) - math.sin(p))
    y = 86.6 * math.sin(p/2)
    z = 53 + 17.7 * (math.cos(p) + math.sin(p))
    return np.array([x,y,z])

def path_p(p):
    x_p = 17.7 * (-math.sin(p) - math.cos(p))
    y_p = 43.3 * math.cos(p/2)
    z_p = 17.7 * (math.cos(p) - math.sin(p))
    return np.array([x_p,y_p,z_p])

def path_pp(p):
    x_pp = 17.7 * (-math.cos(p) + math.sin(p))
    y_pp = -21.65 * math.sin(p/2)
    z_pp = 17.7 * (-math.sin(p) - math.cos(p))
    return np.array([x_pp,y_pp,z_pp])

# def stabilityBasis:
#     e1 = 

#TODO
def C_L(alpha):
    c = 0.15
    return c

def C_D(alpha):
    c = 0.0275
    return c

def R_sc(a):
    a = -a
    return (np.array([[math.cos(a), 0, math.sin(a)],
                      [0, 1, 0],
                      [-math.sin(a), 0, math.cos(a)]]))




### Plot path

In [3]:
ps = np.arange(0,30,0.02)
coords = []
x = []
y = []
z = []
for p in ps:
    coord = path(p)
    coords.append(coord)
    x.append(coord[0])
    y.append(coord[1])
    z.append(coord[2])

# print(ps)
# print(coords)

In [4]:
# %matplotlib widget

current = [2,0,0] # wind in x direction

fig = plt.figure(figsize=(7,7))
ax = plt.axes(projection = '3d')

p = 0.9
p_start = path(p)
pp_start = path_p(p)
print(pp_start)

ax.plot(x, y, z)
ax.plot(p_start[0], p_start[1], p_start[2], 'ro') # start point
ax.quiver(p_start[0], p_start[1], p_start[2], pp_start[0], pp_start[1], pp_start[2], length=0.5, color='r') # tangent at start point
ax.quiver(0,0,0, current[0], current[1], current[2], length=15, color='g')

ax.set_xlim(0, 100)
ax.set_ylim(-100, 100)
ax.set_zlim(0, 100)

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
# plt.ion()
plt.title("Kite path")
plt.show()

<IPython.core.display.Javascript object>

[-24.86738274  38.98935953  -2.86238986]


### test simulation

In [5]:
p0 = 0
pdot0 = 0.1
t_step = 0.02
v_current_i = np.array([1.5,0,0])
rho = 1025
S = 2 #TODO
m = 300
g = 9.81
vol = 1

p = p0
pdot = pdot0

x0 = [p0, pdot0]
t_range = (0, 20)

def kiteDynamics(t, x):
    # print(t)
    p = x[0]
    pdot = x[1]

    r = path(p)
    r_p = path_p(p)
    r_pp = path_pp(p)

    # TODO: pre calculate and use as functions
    # Stability basis
    e1 = r_p / np.linalg.norm(r_p)
    e3 = -r / np.linalg.norm(r)
    e2 = np.linalg.cross(e3,e1) #TODO or reverse?
    e2 /= np.linalg.norm(e2) # Does it need to be normalized?

    # print(e1, e2, e3)

    # Rot matrices 
    # R_si = np.array([e1,e2,e3])
    R_si = np.column_stack([e1, e2, e3])

    # print(R_si)

    # Relative water velocity
    v_kite_i = r_p * pdot
    v_rel_i = v_current_i - v_kite_i

    v_rel_s = R_si.T @ v_rel_i

    alpha_ic = np.atan2(-v_rel_s[2], -v_rel_s[0])# angle between axis e1 and v_rel_s
    # print(v_rel_s)
    v_rel_c = np.array([math.sqrt(v_rel_s[2]**2 + v_rel_s[0]**2), 0, 0])

    v_rel_abs = v_rel_c[0]

    # Forces in current frame
    alpha = alpha_ic #TODO

    F_L = 0.5 * rho * v_rel_abs**2 * S * C_L(alpha)
    F_D = 0.5 * rho * v_rel_abs**2 * S * C_D(alpha)

    F_aero_c = np.array([-F_D, 0, -F_L])

    F_aero_s = R_sc(-alpha_ic).T @ F_aero_c #TODO: check alpha sign -/+??

    F_aero_i = R_si @ F_aero_s

    # Forces in inertial frame
    g_i = np.array([0, 0, -g])
    F_mg = m * g_i
    F_b = rho * vol * g * np.array([0, 0, 1])
    F_tot_i = F_aero_i + F_mg + F_b # thether force unknown here

    # print(F_aero_i, F_mg, F_b)

    #
    pdotdot = (np.dot(e1, F_tot_i) - m * (np.dot(e1, r_pp) * pdot**2)) / (m * np.linalg.norm(r_p))

    F_thether = m * (np.dot(e3, r_pp) * pdot**2) - np.dot(e3, F_tot_i)

    # print(pdotdot, F_thether)

    # print(v_rel_abs, alpha, F_aero_c, F_aero_s, pdot, pdotdot)
    print("\nt:", round(t,4), "p:", round(p,4))
    print(alpha_ic, v_kite_i, v_rel_i, v_rel_s, v_rel_c, v_rel_abs)
    print(F_aero_c, F_aero_s, pdot, pdotdot)
    # print(f"{alpha_ic}\t{v_kite_i}\t{v_rel_i}\t{v_rel_s}\t{v_rel_c}\t{v_rel_abs}")
    # print(f"{F_aero_c}\t{F_aero_s}\t{pdot}\t{pdotdot}")
    # np.linalg.norm(r_p),
    return [pdot, pdotdot]


sol = solve_ivp(kiteDynamics, t_range, x0, max_step = t_step)




t: 0.0 p: 0.0
0.18942236152986888 [-1.77  4.33  1.77] [ 3.27 -4.33 -1.77] [-5.53231374  0.91826178 -1.06066017] [5.63307157 0.         0.        ] 5.63307156925607
[ -894.43152389     0.         -4878.71740305] [   40.1886303      0.         -4959.86654293] 0.1 0.17042891739796526

t: 0.0001 p: 0.0
0.18939352795746792 [-1.77031616  4.33073055  1.7702811 ] [ 3.27031616 -4.33073055 -1.7702811 ] [-5.53316283  0.91826178 -1.06065756] [5.63390498 0.         0.        ] 5.633904981990928
[ -894.69620569     0.         -4880.16112196] [   40.05746998     0.         -4961.33543271] 0.10001687174840333 0.1704185329806561

t: 0.002 p: 0.0002
0.18884730654415408 [-1.77632419  4.34461091  1.77562094] [ 3.27632419 -4.34461091 -1.77562094] [-5.54929552  0.91826178 -1.06060785] [5.64974069 0.         0.        ] 5.649740685403904
[ -899.73287408     0.         -4907.63385864] [   37.55772311     0.         -4989.28639713] 0.10033743496806648 0.17022071615233833

t: 0.003 p: 0.0003
0.1885617705290990

In [6]:
print(sol)
print(sol.t)

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  9.900e-03 ...  1.999e+01  2.000e+01]
        y: [[ 0.000e+00  9.983e-04 ...  9.043e-01  9.042e-01]
            [ 1.000e-01  1.017e-01 ... -4.162e-03 -4.425e-03]]
      sol: None
 t_events: None
 y_events: None
     nfev: 6008
     njev: 0
      nlu: 0
[0.00000000e+00 9.89958081e-03 2.98995808e-02 ... 1.99698996e+01
 1.99898996e+01 2.00000000e+01]


### Plots of motion

In [7]:

import matplotlib.animation as animation

ps = sol.y[0]
pps = sol.y[1]

coords = np.array([path(p) for p in ps])
x = coords[:,0]
y = coords[:,1]
z = coords[:,2]

current = [2,0,0] # wind in x direction

fig = plt.figure(figsize=(7,7))
ax = plt.axes(projection = '3d')

ax.set_xlim(0, 100)
ax.set_ylim(-100, 100)
ax.set_zlim(0, 100)

line, = ax.plot([], [], [], lw=2)
point, = ax.plot([], [], [], 'ro')

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("Kite path")

def init():
    line.set_data([], [])
    line.set_3d_properties([])
    point.set_data([], [])
    point.set_3d_properties([])
    return line, point

def animate(i):
    p = ps[i]
    pdot = pps[i]
    coord = path(p)
    line.set_data(x[:i], y[:i])
    line.set_3d_properties(z[:i])
    point.set_data(coord[0], coord[1])
    point.set_3d_properties(coord[2])
    return line, point


anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(ps), interval=50, blit=False)
plt.show()

anim


<IPython.core.display.Javascript object>

In [8]:
anim.save("animation.mp4", fps=30)

MovieWriter ffmpeg unavailable; using Pillow instead.


IndexError: list index out of range